# Two axes figure example

This example illustrates creating a two-axes layout.


In [ ]:
import numpy as np

from mpllayout import layout as lay
from mpllayout import primitives as pr
from mpllayout import constraints as co
from mpllayout import solver
from mpllayout import ui
from mpllayout import matplotlibutils as lplt

## Specify the layout

In [ ]:
# Create the layout object
layout = lay.Layout()

## Create an origin point fixed at (0, 0)
layout.add_prim(pr.Point(), "Origin")
layout.add_constraint(co.Fix(), ("Origin",), (np.array([0, 0]),))

# Create a box to represent the figure
layout.add_prim(pr.Quadrilateral(), "Figure")
layout.add_constraint(co.Box(), ("Figure",), ())

# Create `Axes` objects to represent the left and right axes
layout.add_prim(pr.Axes(), "AxesLeft")
layout.add_prim(pr.Axes(), "AxesRight")
layout.add_constraint(co.Box(), ("AxesLeft/Frame",), ())
layout.add_constraint(co.Box(), ("AxesRight/Frame",), ())

# Constrain the width and height of the figure
fig_width, fig_height = 6, 3
layout.add_constraint(co.Width(), ("Figure",), (fig_width,))
layout.add_constraint(co.Height(), ("Figure",), (fig_height,))

# Constrain the bottom-left corner of the figure to the origin
layout.add_constraint(co.Coincident(), ("Figure/Line0/Point0", "Origin"), ())

# Constrain the left/right margins to `AxesLeft` and `AxesRight`, respectively
margin_left = 0.5
margin_right = 0.5

layout.add_constraint(
    co.InnerMargin(side='left'), ("AxesLeft/Frame", "Figure",), (margin_left,)
)
layout.add_constraint(
    co.InnerMargin(side='right'), ("AxesRight/Frame", "Figure"), (margin_right,)
)

# Constrain the gap between the left and right axes
margin_inter = 0.5
layout.add_constraint(
    co.OuterMargin(side='right'), ("AxesLeft/Frame", "AxesRight/Frame"), (margin_inter,)
)

# Constrain the top/bottom margins on the left axes ('AxesLeft')
# We can align the left/right axes to implicity set those margins
margin_top = 1.0
margin_bottom = 0.5
layout.add_constraint(
    co.InnerMargin(side="bottom"), ("AxesLeft/Frame", "Figure"), (margin_bottom,)
)
layout.add_constraint(
    co.InnerMargin(side="top"), ("AxesLeft/Frame", "Figure"), (margin_top,)
)

# Align the left/right axes in a row
layout.add_constraint(co.AlignRow(), ("AxesLeft/Frame", "AxesRight/Frame"), ())

# Constrain the width of 'AxesLeft'
# Note that the right axes width is already constrained through the margins and
# known left axes width
width = 2
layout.add_constraint(co.Width(), ("AxesLeft/Frame",), (width,))

## Solve the layout

In [ ]:
# Solve and plot the constrained layout
prims, info = solver.solve(layout, max_iter=40, rel_tol=1e-9)

fig_layout, ax_layout = ui.figure_prims(prims)

## Plot a figure using the layout

In [ ]:
## Create a figure and axes from the constrained primitives
fig, axs = lplt.subplots(prims)

x = np.linspace(0, 1)
axs["AxesLeft"].plot(x, 4 * x)
axs["AxesRight"].plot(x, x**2)

fig.savefig("two_axes.png")
